
1. Euclidean

Given $\phi$ as latitude, $\lambda$ as longitude, $R$ as the earth's radius, and two locations represented by coordinates $A$ and $B$, then:

$$
\begin{aligned}
a & = sin^2(\frac{\phi B-\phi A}{2}) + cos(\phi A) \times cos(\phi B) \times sin^2(\frac{\lambda B - \lambda A}{2}) \\
c & =  2 \times atan2(\sqrt{a}, \sqrt{1-a}) \\
d & = R \times c
\end{aligned}
$$


In [1]:
from collections import namedtuple
from math import radians, sin, cos, atan2, sqrt, degrees

# Point is a convenience wrapper to hold the co-ordinates belonging to
# a particular "point" - i.e. the base station location, or the remote
# device location.
#
# Units: lat, long = radians. alt = meters.
Point = namedtuple('Point', ['lat', 'lon', 'alt'])

def point(lat: float, lon: float, alt: float) -> Point:
  return Point(lat=radians(lat), lon=radians(lon), alt=alt)

# Direction is a convenience wrapper that holds the information about the
# "direction" between two Points; these should translate pretty well for
# use with a 2-axis gimbal.
#
# Units: bearing = degrees. elevation, distance = meters.
Direction = namedtuple('Direction', ['bearing', 'elevation', 'distance'])

class Location:
  def __init__(self, location: Point, precision: int = 2) -> None:
    self.location = location
    self.precision = precision


  def update(self, **kwargs):
    update = { k : kwargs[k] for k in kwargs if k in ['lat', 'lon', 'alt'] }
    if not (len(update) > 0 and len(update) < 3):
      raise ValueError(f"location update must consist of a subset of ['lat', 'lon', 'alt'] - got {list(kwargs.keys())}")

    self.location = self.location._replace(**update)


  @staticmethod
  def _calculate(base: Point, remote: Point, precision: int = 2) -> tuple:
    R = 6371000.0 # Radius of the Earth

    delta_lat = remote.lat - base.lat
    delta_lon = remote.lon - base.lon

    # Distance: Haversine Formula
    distance = R * (2 * atan2(
      sqrt((a := sin(delta_lat / 2)**2 + cos(base.lat) * cos(remote.lat) * sin(delta_lon / 2)**2)),
      sqrt(1 - a)
    ))

    # Angle of Elevation
    angle_of_elevation = degrees(atan2(remote.alt - base.alt, distance))

    # Bearing
    y = sin(delta_lon) * cos(remote.lat)
    x = cos(base.lat) * sin(remote.lat) - (sin(base.lat) * cos(remote.lat) * cos(delta_lon))
    bearing = (degrees(atan2(y, x)) + 360) % 360 # @note: is the degrees() call in the correct location here?

    return Direction(
      bearing   = round(bearing, precision),
      elevation = round(angle_of_elevation, precision),
      distance  = round(distance, precision)
    )


  def direction(self, remote: Point) -> Direction:
    return self._calculate(self.location, remote, self.precision)


## Demonstration

The calculations above can be demonstrated by using `ipyleaflet`; first we need to create:

1. A static marker to represent our "base" location; this will be coloured red.
2. A draggable marker to represent our "remote" location; this will be coloured blue.
3. An input slider that accepts integer values, allowing different altitude values to be used.
4. A HTML widget that can display the results.


In [2]:
from ipyleaflet import Map, Marker, WidgetControl, AwesomeIcon
import ipywidgets as widgets

base_marker = Marker(location=(51.5048226, -0.113632836), icon=AwesomeIcon(
    name='compass',
    marker_color='red',
    icon_color='white',
), draggable=False)

remote_marker = Marker(location=(51.5041468, -0.118133430), icon=AwesomeIcon(
    name='arrows',
    marker_color='blue',
    icon_color='darkblue',
    spin=True
))

altitude_input = widgets.IntSlider(
    value=0,
    min=0,
    max=150,
    step=5,
    description='Altitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

calc_output = widgets.HTML()

A simple function - `handle_updated_parameters` is defined to handle any interactions with the map; this will be triggered if either (a) a new altitude is input, or (b) the "remote" location Marker is moved. 

This function uses the `_calculate(Point, Point)` method that's defined on the `Location` object. The `Point` values are generated directly from the state of the map, and recalculated everytime the function runs. 

In [3]:
def handle_updated_parameters(*args, **kwargs):
    def direction_html(dir):
        out = " - ".join([
            f"<b>Distance (m)</b>: {dir.distance}", f"<b>Elevation</b>: {dir.elevation}&deg;", f"<b>Bearing</b>: {dir.bearing}&deg;"
        ])
        return f"&nbsp;{out}&nbsp;"

    calc_output.value = direction_html(Location._calculate(
        point(base_marker.location[0], base_marker.location[1], 0),
        point(remote_marker.location[0], remote_marker.location[1], altitude_input.value)
    ))

altitude_input.observe(handle_updated_parameters, 'value')
remote_marker.observe(handle_updated_parameters, 'location')
handle_updated_parameters() # handle current/default parameters (sets the output for first load.)

With the event handlers configured, the UI widgets can be added to the map and displayed.

- To adjust the remote location, simply drag the blue marker within the map.
- To adjust the altitude of the remote location, use the slider located at the top-right corner of the map.
- The bearing, angle of elevation, and distance can be found in the bottom-right corner of the map - and they will be automatically updated upon any changes.

In [4]:
m = Map(center=base_marker.location, controls=[], dragging=False, zoom=17)

m.add(base_marker)
m.add(remote_marker)
m.add_control(WidgetControl(widget=altitude_input, position='topright'))
m.add_control(WidgetControl(widget=calc_output, position='bottomright'))

display(m)

Map(center=[51.5048226, -0.113632836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

## Tests

To verify the accuracy of these calculations, a number of simple tests have been included. These are powered by a simple comparison/assertion function, two "runner" functions (`run_geographic_tests` and `run_elevation_tests`), and some `namedtuples` to enhance the readability of the code.

Tests are grouped in to two specific types - largely as throw-back to how code was initially written rather than any specific technical reason. These types are:

- **Geographic Tests:** used to verify that the calculation of bearings and distances are correct.
- **Elevation Tests:** used to verify that the calculation of "angle of elevation" are correct.


In [8]:
from typing import List

def compare(expected: Direction, actual: Direction):
  """ Verifies that all three properties of the output `Direction` are equal to the expected values. """
  return all([
    getattr(expected, prop) == getattr(actual, prop)
    for prop in ["bearing", "elevation", "distance"]
  ])

# Base Location used for performing verification tests; test cases contain the output
# associated with calculations that utilise this location as a base.
TEST_BASE_LOCATION = (51.5069574, -0.112639096)

# RemoteLocation used for performing tests to verify the calculation of "angle of
# elevation" values.
TEST_ALTI_LOCATION = (51.4986765, -0.104676284)

# GeographicTestCase contains a `Point` value, and a `Direction` tuple containing
# the expected Bearing and Distance to that `Point` from TEST_BASE_LOCATION.
GeographicTestCase = namedtuple("GeographicTestCase", ["RemotePoint", "ExpectedDirection"])

# ElevationTestCase contains a base altitude, a remote altitude, and the expected
# "Angle of Elevation" between two points at TEST_BASE_LOCATION and TEST_ALTI_LOCATION
# at those altitudes.
ElevationTestCase = namedtuple("ElevationTestCase", ["BaseAltitude", "RemoteAltitude", "ExpectedElevation"])

def run_geographic_tests(test_cases: List[GeographicTestCase]) -> bool:
  """ Executes a number of list of tests and returns a boolean indicating whether *all* passed. """
  return all([
    compare(t.ExpectedDirection, Location._calculate(point(TEST_BASE_LOCATION[0], TEST_BASE_LOCATION[1], 0), t.RemotePoint))
    for t in test_cases
  ])


def run_elevation_tests(test_cases: List[ElevationTestCase]) -> bool:
  """ Executes a number of list of tests and returns a boolean indicating whether *all* passed. """
  return all([
    compare(
      Direction(149.09, t.ExpectedElevation, 1073.14),
      Location._calculate(point(TEST_BASE_LOCATION[0], TEST_BASE_LOCATION[1], t.BaseAltitude), point(TEST_ALTI_LOCATION[0], TEST_ALTI_LOCATION[1], t.RemoteAltitude))
    ) for t in test_cases
  ])

Using this pattern to produce a simple test suite enables the easy production of "*table driven tests*" - as often seen in Golang. Note that the tests below are limited to distances below approximately 1km - which is the likely upper limit for the specific use-case in mind; but - due to the tabular structure of the tests - it would be trivial to introduce new test cases.

In [12]:
print("Geographic Tests Passing: ", run_geographic_tests([
                  # (Remote Point,                       Expected Direction           )
  GeographicTestCase(point(51.4986765, -0.104676284, 0), Direction(149.09, 0, 1073.14)),
  GeographicTestCase(point(51.4987206, -0.112233761, 0), Direction(178.25, 0, 916.32) ),
  GeographicTestCase(point(51.5008267, -0.119832024, 0), Direction(216.14, 0, 844.14) ),
  GeographicTestCase(point(51.5020523, -0.124047118, 0), Direction(235.37, 0, 959.66) ),
  GeographicTestCase(point(51.5044345, -0.123437039, 0), Direction(249.43, 0, 798.26) ),
  GeographicTestCase(point(51.5072307, -0.121800917, 0), Direction(272.75, 0, 634.81) ),
  GeographicTestCase(point(51.5099059, -0.118168171, 0), Direction(310.59, 0, 503.90) ),
  GeographicTestCase(point(51.5111140, -0.108711939, 0), Direction(30.46,  0, 536.18) ),        
]))

print("Elevation Tests Passing: ", run_elevation_tests([
                   # (Base Altitude,  Remote Altitude, Expected Angle of Elevation)
    ElevationTestCase(-5,             10,              0.80                       ),
    ElevationTestCase( 0,             430,             21.84                      )
]))

Geographic Tests Passing:  True
Elevation Tests Passing:  True
